In [1]:
using AutomotiveDrivingModels
using AutoViz
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using GridInterpolations 

using POMDPs
using POMDPToolbox

20-Sep 15:01:27:WARNING:root:replacing docs for 'AutomotivePOMDPs.scene_to_states :: Tuple{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.
20-Sep 15:01:35:WARNING:root:replacing docs for 'AutomotivePOMDPs.states_to_scene :: Tuple{Dict{Int64,AutomotivePOMDPs.SingleOCState},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.


In [2]:
include("../src/pedestrian_crossing/pomdp_types.jl")
include("../src/pedestrian_crossing/spaces.jl")
include("../src/pedestrian_crossing/transition.jl")
include("../src/pedestrian_crossing/observation.jl")

In [3]:
pomdp = SingleOCFPOMDP()


SingleOCFPOMDP
  env: AutomotivePOMDPs.CrosswalkEnv
  ego_type: AutomotiveDrivingModels.VehicleDef
  ped_type: AutomotiveDrivingModels.VehicleDef
  longitudinal_actions: Array{Float64}((5,)) [1.0, 0.0, -1.0, -2.0, -4.0]
  lateral_actions: Array{Float64}((3,)) [1.0, 0.0, -1.0]
  ΔT: Float64 0.5
  v_noise: Array{Float64}((1,)) [0.0]
  theta_noise: Array{Float64}((1,)) [0.0]
  pos_obs_noise: Float64 1.0
  vel_obs_noise: Float64 1.0
  collision_cost: Float64 -1.0
  action_cost_lon: Float64 0.0
  action_cost_lat: Float64 0.0
  goal_reward: Float64 1.0
  γ: Float64 0.95
  state_space_grid: GridInterpolations.RectangleGrid{6}
  state_space: Array{SingleOCFState}((437500,))
  action_space: Array{SingleOCFAction}((15,))


In [4]:
# Test state space
state_space = states(pomdp)
n_states(pomdp)
x = state_space[state_index(pomdp,SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 1.5) )]
println(x)

cnt = 0
for (index, s) in enumerate(pomdp.state_space)
    idx = state_index(pomdp,s)
    if (idx != index)
        println("error")
        cnt = cnt + 1
    end
end
println(cnt)

SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 1.5)
0


In [5]:
# Test action space
action_space = actions(pomdp)
n_actions(pomdp)
x = action_space[action_index(pomdp,SingleOCFAction(-4.0, 0.0) )]
println(x)

cnt = 0
for (index, a) in enumerate(pomdp.action_space)
    idx = action_index(pomdp,a)
    if (idx != index)
        println("error")
        cnt = cnt + 1
    end
end
println(cnt)

SingleOCFAction(-4.0, 0.0)
0


In [6]:
# Test transition
#=
    ego_y::Float64
    ego_v::Float64    
    ped_s::Float64
    ped_T::Float64
    ped_theta::Float64
    ped_v::Float64
=#
state = SingleOCFState(0.0, 0.0, 30.0, 3.0, 1.57, 0.0)
action = SingleOCFAction(0.0, 0.0)

tic()
x = transition(pomdp,state,action,1.0)
println(x)
toc()

POMDPToolbox.SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[SingleOCFState(0.0, 0.0, 30.25, 2.77778, 1.57, 0.0), SingleOCFState(0.0, 0.0, 30.25, 3.88889, 1.57, 0.0)], [0.615385, 0.384615])
elapsed time: 1.944565969 seconds


1.944565969

In [7]:
# Test observation space

observation_space = observations(pomdp)
n_observations(pomdp)
idx = obs_index(pomdp,SingleOCFObs(0.0, 0.0, 49.0, -5.0, 1.57, 1.5))
x = observation_space[idx]
println(x)

SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 1.5)


In [8]:
# Test observation
observ = SingleOCFObs(0.0, 0.0, 30.0, 3.0, 1.57, 1.5)
action = SingleOCFAction(0.0, 0.0)
observation(pomdp,action,observ)

POMDPToolbox.SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[SingleOCFState(0.0, 0.0, 30.0, 3.0, 1.57, 1.5)], [1.0])

In [9]:
# Test collision checker
collision_checker(pomdp,SingleOCFState(0.0,0.0,1.0,0.0,0.0,0.0))


true

In [10]:
# Test isterminal
isterminal(pomdp, SingleOCFState(0.0,50.0,10.0,0.0,0.0,0.0))


false

In [11]:
# Simulate
s0 = SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 0.5)

sim(pomdp, s0, initial_obs=s0, max_steps=10) do s
    println("state is: $s")
    a = SingleOCFAction(1.0, 0.0)
    println("moving $a")
    return a
end;


state is: SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 1.0769230769230769, 49.0, -3.888888888888889, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 2.1538461538461537, 49.0, -2.7777777777777777, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 2.1538461538461537, 49.0, -1.6666666666666667, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 2.1538461538461537, 46.916666666666664, -1.6666666666666667, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 2.1538461538461537, 44.833333333333336, -1.6666666666666667, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 2.1538461538461537, 42.75, -0.5555555555555556, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFState(0.0, 3.230769230769231, 40.666666666666664, -0.5555555555555556, 1.57, 0.5)
moving SingleOCFAction(1.0, 0.0)
state is: SingleOCFSt

In [12]:
# Simulate to prefer
#=
    ego_y::Float64
    ego_v::Float64    
    ped_s::Float64
    ped_T::Float64
    ped_theta::Float64
    ped_v::Float64
=#

hr = HistoryRecorder(max_steps = 10)
policy = RandomPolicy(pomdp)


s0 = SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 0.0)
h = simulate(hr, pomdp, policy, updater(policy), nothing, s0)
h.state_hist

11-element Array{SingleOCFState,1}:
 SingleOCFState(0.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(0.5, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(0.5, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(0.5, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(0.5, 0.0, 49.0, -5.0, 1.57, 0.0)
 SingleOCFState(1.0, 0.0, 49.0, -5.0, 1.57, 0.0)

In [13]:
using QMDP

solver = QMDPSolver(max_iterations=50, tolerance=1e-3) 
# run the solver
qmdp_policy = solve(solver, pomdp, verbose=true)


20-Sep 15:02:01:WARNING:root:Keyword args for solve(::QMDPSolver, ::POMDP) are no longer supported. For verbose output, use the verbose option in the ValueIterationSolver


POMDPToolbox.AlphaVectorPolicy{SingleOCFPOMDP,SingleOCFAction}(SingleOCFPOMDP
  env: AutomotivePOMDPs.CrosswalkEnv
  ego_type: AutomotiveDrivingModels.VehicleDef
  ped_type: AutomotiveDrivingModels.VehicleDef
  longitudinal_actions: Array{Float64}((5,)) [1.0, 0.0, -1.0, -2.0, -4.0]
  lateral_actions: Array{Float64}((3,)) [1.0, 0.0, -1.0]
  ΔT: Float64 0.5
  v_noise: Array{Float64}((1,)) [0.0]
  theta_noise: Array{Float64}((1,)) [0.0]
  pos_obs_noise: Float64 1.0
  vel_obs_noise: Float64 1.0
  collision_cost: Float64 -1.0
  action_cost_lon: Float64 0.0
  action_cost_lat: Float64 0.0
  goal_reward: Float64 1.0
  γ: Float64 0.95
  state_space_grid: GridInterpolations.RectangleGrid{6}
  state_space: Array{SingleOCFState}((437500,))
  action_space: Array{SingleOCFAction}((15,))
, Array{Float64,1}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0